<a href="https://colab.research.google.com/github/joaomj/tipos_distancias/blob/master/tipos_distancias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Orientações gerais**
- Este código tem fins educacionais apenas.
- Você precisa criar um dataset de exemplo para testar esse código. Futuramente eu vou implementar essa função aqui.
- Cuidado com o excesso de requisições às APIs. Eu testei com 200 ceps distintos e demorei quase 2h para obter as distâncias.

# **Imports**

In [ ]:
import pandas as pd
import os
import numpy as np
import warnings
import time
import requests
import csv
import random
import string

In [ ]:
# Configurações gerais

# determinando que ele não exiba os dados em formato notação científica
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Configuração para exibir apenas 2 dígitos após a vírgula
pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# **Funções**

In [ ]:
# -----------------------------------------
# Função para escolher aleatoriamente ceps de uma lista de ceps válidos
def generate_random_cep():
    ceps = ['14875-530', '69315-228', '83203-716', '09432-115'] # exemplos de ceps válidos

    # Seleciona aleatoriamente da lista de CEPs
    selected_cep = random.choice(ceps)

    return selected_cep

# -----------------------------------------
# Função para gerar um dataset de exemplo
def generate_dataset(n):

    # Gera dados de exemplo
    data = []

    for _ in range(n):
        sku = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
        origin = generate_random_cep()
        destination = generate_random_cep()
        data.append([sku, origin, destination])

    # Escreve os dados no arquivo CSV
    with open('dados.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['sku', 'origin', 'destination'])  # Escrever o cabeçalho
        writer.writerows(data)  # Escrever as linhas

    print("Arquivo CSV gerado com sucesso!")

    # Obtendo o path deste notebook
    current_dir = os.getcwd()

    # Importando arquivos
    df = pd.read_csv('dados.csv')
    return df

# -------------------------------------------
# Função para obter coordenadas de um CEP usando a API, com rate limiting
def get_coordinates(cep):
    url = f'https://cep.awesomeapi.com.br/json/{cep}'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return float(data['lat']), float(data['lng'])
        else:
            return np.nan, np.nan
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {cep}: {e}")
        return np.nan, np.nan
    finally:
        # Esperar 1 segundo entre as requisições para não sobrecarregar a API
        time.sleep(1)

# -------------------------------------------
# Função para obter a distância de condução entre duas coordenadas com rate limit e cache
def get_driving_distance(origin_coords, dest_coords):

    # Declarar o cache de distâncias
    distance_cache = {}

    # Verificar se a distância já foi calculada e está no cache
    if (origin_coords, dest_coords) in distance_cache:
        return distance_cache[(origin_coords, dest_coords)]

    profile = "driving"  # Tipo de perfil, pode ser driving, walking, cycling etc.
    coordinates = f"{origin_coords[1]},{origin_coords[0]};{dest_coords[1]},{dest_coords[0]}"
    url = f"http://router.project-osrm.org/route/v1/{profile}/{coordinates}?overview=false&alternatives=false&steps=false&annotations=false"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'routes' in data and len(data['routes']) > 0:
            distance_km = data['routes'][0]['distance'] / 1000  # Convertendo de metros para quilômetros
            distance_cache[(origin_coords, dest_coords)] = distance_km
            time.sleep(1)  # Intervalo de 1 segundo entre as requisições
            return distance_km

    return np.nan

# -------------------------------------------
# Função para obter a distância do dicionário
def get_distance(row):
    return cep_distances.get((row['origin'], row['destination']), 'Distância não encontrada')

# **Loading data**

In [ ]:
df = generate_dataset(2) #dataset com 2 linhas
df.head()

Arquivo CSV gerado com sucesso!


,sku,origin,destination
0,ZMF8MDD4,83203-716,14875-530
1,0T9NT3W5,69315-228,14875-530


# **Obtendo as distâncias**
Campos 'origin' e 'destination' são CEPs. Alguns ceps que deveriam começar com zero estão sem esses números, sendo necessário formatá-los para que todos os ceps tenham 8 dígitos.

In [ ]:
df['origin'].unique()

array(['83203-716', '69315-228'], dtype=object)

In [ ]:
df['destination'].unique()

array(['14875-530'], dtype=object)

###Cálculo das distâncias de **deslocamento** entre os ceps, **em km:**

In [ ]:
# colunas 'origin' e 'destination' têm os CEPs
unique_origins = df['origin'].unique()
unique_destinations = df['destination'].unique()

# Dicionário para armazenar as distâncias entre os CEPs
cep_distances = {}

# Iterar sobre os pares únicos de origem e destino
for origin_cep in unique_origins:
    # Obter coordenadas do CEP de origem
    origin_coords = get_coordinates(origin_cep)
    if np.isnan(origin_coords).any():
        print('CEP de origem inválido')
        break

    for dest_cep in unique_destinations:
        # Obter coordenadas do CEP de destino
        dest_coords = get_coordinates(dest_cep)
        if np.isnan(dest_coords).any():
            print('CEP de destino inválido')
            break

        # Calcular a distância de condução entre as coordenadas
        distance = get_driving_distance(origin_coords, dest_coords)

        # Armazenar a distância no dicionário
        cep_distances[(origin_cep, dest_cep)] = distance


In [ ]:
# grava distâncias no dataframe
df['distancia_km'] = df.apply(get_distance, axis=1)
df.head()

,sku,origin,destination,distancia_km
0,ZMF8MDD4,83203-716,14875-530,716.05
1,0T9NT3W5,69315-228,14875-530,4329.49
